In [ ]:
import os
import pickle
import PIL.Image
import numpy as np
import dnnlib
import dnnlib.tflib as tflib
import config
from encoder.generator_model import Generator

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import pretrained_networks
blended_url = './checkpoints/us_comics/000036/blended-128.pkl'
ffhq_url = './weights/stylegan2-ffhq-config-f.pkl'
_, _, Gs_blended = pretrained_networks.load_networks(blended_url)
_, _, Gs = pretrained_networks.load_networks(ffhq_url)

In [ ]:
from PIL import Image

def generate_image(latent_vector):

    latent = np.expand_dims(latent_vector, axis=0)
    synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=False),
                                    minibatch_size=8)
    tar_images = Gs_blended.components.synthesis.run(latent, randomize_noise=False, **synthesis_kwargs)
    ori_images = Gs.components.synthesis.run(latent, randomize_noise=False, **synthesis_kwargs)
    tar_images = Image.fromarray(tar_images.transpose((0, 2, 3, 1))[0], 'RGB')
    ori_images = Image.fromarray(ori_images.transpose((0, 2, 3, 1))[0], 'RGB')
    return ori_images, tar_images

def move_and_show(latent_vector, direction, coeffs, save_name):
    fig,ax = plt.subplots(1, len(coeffs), figsize=(15, 10), dpi=80)
    for i, coeff in enumerate(coeffs):
        new_latent_vector = latent_vector.copy()
        new_latent_vector[:8] = (latent_vector + coeff*direction)[:8]
        ori_images, tar_images = generate_image(new_latent_vector)
        ax[i].imshow(tar_images)
        ax[i].set_title('Ori coeff: %0.1f' % coeff)
        ori_images.save(f"/localhome/local-jackyyin/Style-Transfer-Gan-with-Few-shot-Data/style-transfer-gan/final_vis/us_comics/{save_name}_ori_{i}.png")
        tar_images.save(f"/localhome/local-jackyyin/Style-Transfer-Gan-with-Few-shot-Data/style-transfer-gan/final_vis/us_comics/{save_name}_tar_{i}.png")
#         ax[2*i].imshow(tar_images)
#         ax[2*i].set_title('Tar Coeff: %0.1f' % coeff)
    [x.axis('off') for x in ax]
    plt.show()

In [ ]:
# Loading already learned representations
andrew = np.load('/localhome/local-jackyyin/Style-Transfer-Gan-with-Few-shot-Data/style-transfer-gan/generated/celebrity/20241126-180511_01.npy')
donald = np.load('/localhome/local-jackyyin/Style-Transfer-Gan-with-Few-shot-Data/style-transfer-gan/generated/celebrity/20241126-180845_01.npy')
harris = np.load('/localhome/local-jackyyin/Style-Transfer-Gan-with-Few-shot-Data/style-transfer-gan/generated/celebrity/20241126-180825_01.npy')
jennier = np.load('/localhome/local-jackyyin/Style-Transfer-Gan-with-Few-shot-Data/style-transfer-gan/generated/celebrity/20241126-180943_01.npy')

# Of course you can learn your own vectors using two scripts

# 1) Extract and align faces from images
# python align_images.py raw_images/ aligned_images/

# 2) Find latent representation of aligned images
# python encode_images.py aligned_images/ generated_images/ latent_representations/

In [ ]:
# Loading already learned latent directions
smile_direction = np.load('ffhq_dataset/latent_directions/smile.npy')
gender_direction = np.load('ffhq_dataset/latent_directions/gender.npy')
age_direction = np.load('ffhq_dataset/latent_directions/age.npy')

# In general it's possible to find directions of almost any face attributes: position, hair style or color ... 
# Additional scripts for doing so will be realised soon

# Smile transformation

In [ ]:
move_and_show(donald, smile_direction, [-5, 0, 10], 'donald')
move_and_show(andrew, smile_direction, [-5, 0, 10], 'andrew')
move_and_show(harris, smile_direction, [-5, 0, 10], 'harris')
move_and_show(jennier, smile_direction, [-5, 0, 10], 'jennier')


In [ ]:
move_and_show(hillary_clinton, smile_direction, [-1, 0, 1])

# Gender transformation

In [ ]:
move_and_show(donald_trump, gender_direction, [-2, 0, 2])

In [ ]:
move_and_show(hillary_clinton, gender_direction, [-1.5, 0, 1.])

# Age transformation

In [ ]:
move_and_show(hillary_clinton, age_direction, [-2, 0, 2])

In [ ]:
# try it yourself!
move_and_show(donald_trump, age_direction, [-3, 0, 1])